# Imports

In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Load the Pandas libraries with alias 'pd'
import datetime

import numpy as np
import pandas as pd
from bokeh.io import output_notebook
from bokeh.layouts import grid, gridplot
from bokeh.models import BooleanFilter, CDSView, ColumnDataSource, LinearAxis, Range1d, Span
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10_6, Category20_20
from bokeh.plotting import figure, show

import math

from sklearn.linear_model import LinearRegression

from covid19.covid import (get_all_states_data, get_brazil_data,
                           get_date_date_cases_greater_than,
                           get_dia_de_contaminacao_array)
from covid19.scrap import get_covid_data, read_datafile_from_disc

output_notebook(hide_banner=True)

import logging

logging.getLogger().setLevel(logging.INFO)

In [7]:
import qgrid

# Lendo os dados

In [8]:
# data = pd.read_excel("DT_PAINEL_COVIDBR_20200510.xlsx")
# data = read_datafile_from_disc("DT1_PAINEL_COVIDBR_20200512.xlsx")
data = get_covid_data()

# Note: If this starts to sleep to many times stop the execution and run the cell again

INFO:root:Downloading data from the internet
INFO:root:The download was finished!


# Entendendo os dados e separando dados do Brasil todo e de cada estado

As primeiras linhas possuem dados para o Brasil todo e a região é "Brasil".

In [9]:
print("Shape do dataset todo", data.shape)
qgrid.show_grid(data, grid_options=dict(maxVisibleRows=5))

Shape do dataset todo (138635, 17)


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Note que a função que lê os dados adicionou uma coluna contendo o dia da semana.

Vamos separar o dataset em dois: um para o Brasil todo e outro para os estados. Note que colunas irrelevantes (como "município", por exemplo) foram removidas e 4 colunas novas foram adicionadas: **casosNovos**, **obitosNovos**, **diasDeContaminacao_1** e **diasDeContaminacao_100**.

In [10]:
data_brasil = get_brazil_data(data)
print("Shape do dataset do Brasil", data_brasil.shape)
qgrid.show_grid(data_brasil, grid_options=dict(maxVisibleRows=10))

Shape do dataset do Brasil (96, 12)


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [11]:
data_estados = get_all_states_data(data)
print("Shape do dataset dos estados", data_estados.shape)
print(data_estados.shape[0] / 77)

Shape do dataset dos estados (2139, 15)
27.77922077922078


In [12]:
# data_estados.groupby(by="estado").count()
qgrid.show_grid(data_estados, grid_options=dict(maxVisibleRows=10))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Primeiros casos

In [13]:
# data_brasil = data.groupby("data").sum()

print(f"Data do primeiro caso: {get_date_date_cases_greater_than(data, 0)}")
print(
    f"Data com mais de 10 casos: {get_date_date_cases_greater_than(data, 10)}")
print(
    f"Data com mais de 100 casos: {get_date_date_cases_greater_than(data, 100)}"
)
print(
    f"Data com mais de 1000 casos: {get_date_date_cases_greater_than(data, 1000)}"
)

Data do primeiro caso: 2020-02-26
Data com mais de 10 casos: 2020-03-06
Data com mais de 100 casos: 2020-03-14
Data com mais de 1000 casos: 2020-03-21


In [14]:
qgrid.show_grid(data_estados[data_estados.estado == "CE"],
                grid_options=dict(maxVisibleRows=5))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Plot dados para o Brasil

In [15]:
source_brasil = ColumnDataSource(data_brasil)
fig = figure(title="Covid-19 no Brasil",
             x_axis_type='datetime',
             width=1200,
             height=400,
             x_axis_label="Data",
             y_axis_label="Casos Acumulados")
fig.line("data",
         "casosAcumulado",
         source=source_brasil,
         color=Category10_6[0],
         legend_label="Acumulados")
fig.circle("data",
           "casosAcumulado",
           source=source_brasil,
           color=Category10_6[0],
           legend_label="Acumulados")
fig.line("data",
         "obitosAcumulado",
         source=source_brasil,
         color=Category10_6[1],
         legend_label="Óbitos")
fig.circle("data",
           "obitosAcumulado",
           source=source_brasil,
           color=Category10_6[1],
           legend_label="Óbitos")
fig.legend.location = "top_left"
fig.legend.click_policy = 'hide'
fig.add_tools(
    HoverTool(
        tooltips=[("Casos Acumulados",
                   "@casosAcumulado"), ("Óbitos Acumulados", "@obitosAcumulado"
                                       ), ("Semana Epidemiológica",
                                           "@semanaEpi")]))

show(fig)

## Plot de vários estados por data

In [16]:
# state_abrv_vec = np.array(['CE', 'DF', 'PE', 'RJ', 'SP', 'BA', 'AM', 'MS', 'RN'])
state_abrv_vec = np.array(['CE', 'DF', 'RJ', 'SP', 'AM'])


def get_plot(column_name, y_axis_label="Label", width=1200, height=400):

    fig = figure(title="Covid-19 nos estados",
                 x_axis_type='datetime',
                 width=width,
                 height=height,
                 x_axis_label="Data",
                 y_axis_label=y_axis_label)
    for i, sigla in enumerate(state_abrv_vec):
        state_data = data_estados[data_estados.estado == sigla]
        source = ColumnDataSource(state_data)
        fig.line("data",
                 column_name,
                 source=source,
                 legend_label=sigla,
                 color=Category10_6[i])
        fig.circle("data",
                   column_name,
                   source=source,
                   legend_label=sigla,
                   color=Category10_6[i])

    fig.legend.location = "top_left"
    #     fig.legend.title = "Estado"
    fig.legend.click_policy = 'hide'
    fig.add_tools(
        HoverTool(
            tooltips=[("Estado", "@estado"), ("Dia da Semana", "@diaDaSemana"),
                      ("Casos Acumulados",
                       "@casosAcumulado"), ("Casos Novos", "@casosNovos"),
                      ("Óbitos Acumulados",
                       "@obitosAcumulado"), ("Óbitos Novos", "@obitosNovos")]))
    return fig


# xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
width = 650
height = 300
plot_casosAcumulados = get_plot("casosAcumulado",
                                y_axis_label="Casos Acumulados",
                                width=width,
                                height=height)
plot_casosNovos = get_plot("casosNovos",
                           y_axis_label="Casos Novos",
                           width=width,
                           height=height)
plot_casosNovos.x_range = plot_casosAcumulados.x_range
plot_obitosAcumulados = get_plot("obitosAcumulado",
                                 y_axis_label="Óbitos Acumulados",
                                 width=width,
                                 height=height)
plot_obitosAcumulados.x_range = plot_casosAcumulados.x_range
plot_obitosNovos = get_plot("obitosNovos",
                            y_axis_label="Óbitos Novos",
                            width=width,
                            height=height)
plot_obitosNovos.x_range = plot_casosAcumulados.x_range

show(
    gridplot([[plot_casosAcumulados, plot_casosNovos],
              [plot_obitosAcumulados, plot_obitosNovos]]))

## Plot de vários estados por dia de contaminação


In [17]:
# Estados com crescimento mais rápido por dia de contaminação
estados_crescimento_rapido = ["CE", "SP", "AM", "PE", "RJ"]


def get_plot_por_dia_de_contaminacao(
    column_name,
    y_axis_label="Label",
    min_casos=1,  # Deve ser ou 1 ou 100
    width=1200,
    height=400,
    title="Covid-19 nos 5 estados com crescimento mais rápido por dia de contaminação"
):
    fig = figure(title=title,
                 width=width,
                 height=height,
                 x_axis_label="Dia de Contaminação",
                 y_axis_label=y_axis_label)

    for i, sigla in enumerate(estados_crescimento_rapido):
        state_data = data_estados[data_estados.estado == sigla]
        source = ColumnDataSource(state_data)
        fig.line(f"diasDeContaminacao_{min_casos}",
                 column_name,
                 source=source,
                 legend_label=sigla,
                 color=Category20_20[i % 20])
        fig.circle(f"diasDeContaminacao_{min_casos}",
                   column_name,
                   source=source,
                   legend_label=sigla,
                   color=Category20_20[i % 20])

    fig.legend.location = "top_left"
    #     fig.legend.title = "Estado"
    fig.legend.click_policy = 'hide'
    fig.add_tools(
        HoverTool(tooltips=[("Estado", "@estado"),
                            ("Dias de contaminação",
                             f"@diasDeContaminacao_{min_casos}"),
                            ("Data", "@data{%F}"),
                            ("Dia da Semana", "@diaDaSemana"),
                            ("Casos Acumulados", "@casosAcumulado"),
                            ("Casos Novos", "@casosNovos"),
                            ("Óbitos Acumulados", "@obitosAcumulado"),
                            ("Óbitos Novos", "@obitosNovos")],
                  formatters={"@data": "datetime"}))
    return fig


plot_casosAcumulados_por_dia_de_contaminacao_1 = get_plot_por_dia_de_contaminacao(
    "casosAcumulado",
    y_axis_label="Casos Acumulados",
    min_casos=1,
    width=width,
    height=int(1.3 * height),
    title="Covid-19 por dia de contaminação (desde o primeiro caso)")
plot_casosAcumulados_por_dia_de_contaminacao_100 = get_plot_por_dia_de_contaminacao(
    "casosAcumulado",
    y_axis_label="Casos Acumulados",
    min_casos=100,
    width=width,
    height=int(1.3 * height),
    title="Covid-19 por dia de contaminação (desde o centésimo caso)")
# plot_casosNovos_por_dia_de_contaminacao = get_plot_por_dia_de_contaminacao("casosNovos", y_axis_label="Casos Novos", width=width, height=height)
show(
    gridplot([[
        plot_casosAcumulados_por_dia_de_contaminacao_1,
        plot_casosAcumulados_por_dia_de_contaminacao_100
    ]]))
# show(plot_casosAcumulados_por_dia_de_contaminacao_100)

# Verificando se estamos saindo da crise

Um bom gráfico para saber se estamos saindo da crise é o de **casos novos por casos existentes** (ambos em escala log). Enquanto estivermos na epidemia o gráfico aproxima-se de uma reta. Quando começarmos a sair da epidemia o gráfico cai.

In [18]:
def get_plot_casos_novos_por_casos_acumulados(
        df,
        width=1200,
        height=250,
        title="Casos novos por casos acumulados",
        vertical_line_dates=None):
    fig = figure(title=title,
                 width=width,
                 height=height,
                 x_axis_type="log",
                 y_axis_type="log",
                 x_axis_label="Casos Acumulados (log)",
                 y_axis_label="Casos Novos (log)")
    # We will give a "name" to the plot such that in the Hovertool we can set the hover only for the plot (and not for the regression)
    name = "plot"
    fig.line('casosAcumulado',
             "casosNovos",
             source=df,
             name=name,
             legend_label="Casos Novos por Acumulados")
    fig.circle('casosAcumulado',
               "casosNovos",
               source=df,
               name=name,
               legend_label="Casos Novos por Acumulados")
    fig.add_tools(
        HoverTool(names=[name],
                  tooltips=[("Data", "@data{%F}"),
                            ("Dia da Semana", "@diaDaSemana"),
                            ("Casos Acumulados", "@casosAcumulado"),
                            ("Casos Novos", "@casosNovos"),
                            ("Óbitos Acumulados", "@obitosAcumulado"),
                            ("Óbitos Novos", "@obitosNovos")],
                  formatters={"@data": "datetime"}))

    # xxxxxxxxxx Linear prediction xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
    # Index of when we reached 100 cases
    first_index = np.argmax(df["casosAcumulado"] > 100)

    log_novos = np.log10(df["casosNovos"][first_index:])
    log_acumulados = np.log10(df["casosAcumulado"][first_index:])

    model = LinearRegression()
    model.fit(log_acumulados[:, np.newaxis], log_novos)

    min_value = log_acumulados.iloc[0]
    max_value = log_acumulados.iloc[-1]

    x_pred = np.linspace(min_value, max_value, 50)[:, np.newaxis]
    y_pred = model.predict(x_pred)
    fig.line(10**x_pred.flatten(),
             10**y_pred,
             color="red",
             name="regression",
             legend_label="Regressão")
    #     fig.circle(10**x_pred.flatten(), 10**y_pred, color="red", name="regression", legend_label="Regressão")
    fig.add_tools(
        HoverTool(names=["regression"],
                  tooltips=[("Casos Acumulados", "@x{i}"),
                            ("Casos Novos (Regressão)", "@y{i}")]))

    fig.legend.location = "top_left"
    # xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

    if vertical_line_dates is not None:
        for casosAcumulados in vertical_line_dates:
            vertical_line = Span(location=casosAcumulados,
                                 dimension='height',
                                 line_color='green',
                                 line_dash='solid',
                                 line_width=0.5)
            fig.add_layout(vertical_line)

    return fig


plot_casos_novos_por_acumulados_brasil = get_plot_casos_novos_por_casos_acumulados(
    data_brasil, title="Casos novos por casos acumulados (Brasil)", width=width)

data_ce = data_estados[data_estados.estado == "CE"]
plot_casos_novos_por_acumulados_ce = get_plot_casos_novos_por_casos_acumulados(
    data_ce,
    title="Casos novos por casos acumulados (CE)",
    width=width,
    vertical_line_dates=[35122])

data_sp = data_estados[data_estados.estado == "SP"]
plot_casos_novos_por_acumulados_sp = get_plot_casos_novos_por_casos_acumulados(
    data_sp, title="Casos novos por casos acumulados (SP)", width=width)

data_pe = data_estados[data_estados.estado == "PE"]
plot_casos_novos_por_acumulados_pe = get_plot_casos_novos_por_casos_acumulados(
    data_pe, title="Casos novos por casos acumulados (PE)", width=width)

data_rj = data_estados[data_estados.estado == "RJ"]
plot_casos_novos_por_acumulados_rj = get_plot_casos_novos_por_casos_acumulados(
    data_rj, title="Casos novos por casos acumulados (RJ)", width=width)

data_am = data_estados[data_estados.estado == "AM"]
plot_casos_novos_por_acumulados_am = get_plot_casos_novos_por_casos_acumulados(
    data_am, title="Casos novos por casos acumulados (AM)", width=width)

show(
    gridplot([[
        plot_casos_novos_por_acumulados_brasil,
        plot_casos_novos_por_acumulados_ce
    ], [plot_casos_novos_por_acumulados_sp, plot_casos_novos_por_acumulados_rj],
              [
                  plot_casos_novos_por_acumulados_pe,
                  plot_casos_novos_por_acumulados_am
              ]]))

Podemos ver que **por volta do dia 23 de Maio, quando tínhamos 35122 casos acumulados, o Ceará começa a ficar consistentemente abaixo da curva de regressão** indicando que a velocidade de crescimento dos novos casos está caindo.

Segundo a PUC ([ver reportagem](https://diariodonordeste.verdesmares.com.br/editorias/metro/ce-tem-menor-taxa-de-contagio-do-pais-e-estabiliza-curva-de-covid-19-1.2250545)), no dia 25 de Maio a taxa de contágio no Ceará finalmente ficou abaixo de $1$, com o valor de $0.99$.

## Vamos ver em quais dias da semana tem mais casos

In [19]:
group = data_ce.groupby("diaDaSemana").sum().reset_index()
df3 = pd.DataFrame(group)

# Não precisamos fazer um sort no dataframe para os dias ficarem em ordem.
# Ao invés disso vamos criar uma lista com os dias na ordem que queremos e usar essa lista como x_range no bokeh.
dias = ["Domingo", "Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado"]

fig = figure(
    x_range=dias,
    plot_width=800,
    plot_height=400,
    title="Total de casos novos por dia da semana no Ceará",
    #              toolbar_location=None,
    #              tools="",
    x_axis_label="Dia da Semana",
    y_axis_label="Total de Casos Novos")

fig.vbar(x="diaDaSemana", top="casosNovos", width=0.7, source=df3)
fig.add_tools(HoverTool(tooltips=[("Casos Novos", "@casosNovos")]))
show(fig)